In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Load cats_vs_dogs dataset from TensorFlow Datasets

In [ ]:
import os
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
import zipfile



with zipfile.ZipFile(r"G:\MY PROJECTS\TENSORFLOW\Transfer Learning with MobileNet\kagglecatsanddogs_5340.zip", 'r') as zip_ref:

    zip_ref.extractall()
from os import listdir
listdir(r"G:\MY PROJECTS\TENSORFLOW\Transfer Learning with MobileNet\PetImages")
#filter out corrupted images
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} images.")

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

data_dir = r"G:\MY PROJECTS\TENSORFLOW\Transfer Learning with MobileNet\PetImages"
batch_size = 32
img_size = (160, 160)
seed = 123

# Load training and validation datasets from the filtered local directory
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

# Preprocess: resize and normalize

In [ ]:
IMG_SIZE = 160
def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))     # Resize to 160x160
    image = image / 255.0                                    # Normalize to [0, 1]
    return image, label
 
train_ds = train_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(format_image).batch(32).prefetch(tf.data.AUTOTUNE)

# Load MobileNetV2 as feature extractor (frozen)

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_url, input_shape=(IMG_SIZE, IMG_SIZE, 3), trainable=False
)

# Build the model

In [ ]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = feature_extractor_layer(inputs)
print(x.shape)  # Should be (None, 1280)
# Remove any shape or flatten logic, connect directly to Dense
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=3)

# Show prediction on one image

In [ ]:
for image, label in val_ds.take(1):
    pred = model.predict(image[:1])
    plt.imshow(image[0])
    plt.title(f"Predicted: {'Dog' if pred[0][0] > 0.5 else 'Cat'}")
    plt.axis('off')
    plt.show()
    break